In [92]:
%matplotlib inline

import requests
import shutil
import json
import pandas as pd
import matplotlib.pyplot as plt

## Load the Twitter Trend Snapshots

Take the raw fata from dynamoDB dump, clean it up, and load it into a pandas data frame.

In [2]:
with open('output.json', 'rb') as f:
    trends = json.load(f)

In [3]:
# use this to inform cleanup process
trends["Items"][0]

{u'placeID': {u'N': u'2407517'},
 u'trend': {u'M': {u'date': {u'S': u'2016-03-03'},
   u'mappedName': {u'S': u'world book day'},
   u'placeName': {u'S': u'Fresno'},
   u'tweetName': {u'S': u'#WorldBookDay'},
   u'tweetVolume': {u'N': u'225775'},
   u'url': {u'S': u'http://twitter.com/search?q=%23WorldBookDay'}}},
 u'trendTime': {u'N': u'1457022783'}}

In [4]:
trends_clean = []

In [5]:
for trend in trends["Items"]:
    new = {}
    new["placeID"] = trend["placeID"]["N"]
    new["trendMapped"] = trend["trend"]["M"]["mappedName"]["S"]
    new["placeName"] = trend["trend"]["M"]["placeName"]["S"]
    new["trendName"] = trend["trend"]["M"]["tweetName"]["S"]
    new["trendVolume"] = trend["trend"]["M"]["tweetVolume"]["N"]
    new["searchUrl"] = trend["trend"]["M"]["url"]["S"]
    new["trendDate"] = trend["trend"]["M"]["date"]["S"]
    new["trendTime"] = trend["trendTime"]["N"]
    trends_clean.append(new)

In [6]:
# it worked
trends_clean[0]

{'placeID': u'2407517',
 'placeName': u'Fresno',
 'searchUrl': u'http://twitter.com/search?q=%23WorldBookDay',
 'trendDate': u'2016-03-03',
 'trendMapped': u'world book day',
 'trendName': u'#WorldBookDay',
 'trendTime': u'1457022783',
 'trendVolume': u'225775'}

In [7]:
df_trends = pd.DataFrame(trends_clean)

## Pandas Cleanup
Converting columns and shit

In [8]:
df_trends.trendTime = df_trends.trendTime.astype(int)
df_trends.trendVolume = df_trends.trendVolume.astype(int)
df_trends.index = pd.to_datetime(df_trends.trendTime, unit = 's')
df_trends = df_trends.drop("trendTime", axis = 1)

In [142]:
trends_agg_322 = df_trends[(df_trends["trendDate"] == '2016-04-17') & ~(df_trends["trendName"].str.contains("Leicester|ALDUB|iHeartAwards"))].groupby(["placeName","trendName"]).max()

In [10]:
test = trends_agg_322["trendVolume"].groupby(level=0, group_keys=False)

In [11]:
res = test.apply(lambda x: x.sort_values(ascending=False).head(3))

In [143]:
trends_agg_322 = trends_agg_322.sort_values("trendVolume", ascending=False)

In [144]:
trends_agg_322 = df_trends[~(df_trends["trendName"].str.contains("Leicester|ALDUB|iHeartAwards"))].sort_values("trendVolume", ascending=False).groupby(["placeName","trendDate","trendName"]).max()

In [146]:
test = trends_agg_322["trendVolume"].groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(3))

In [147]:
trends_agg_322.tail(50)

placeID  \
placeName  trendDate  trendName                             
Washington 2016-04-09 Bruce Springsteen           2514815   
                      Trevor Story                2514815   
           2016-04-10 #EverydayKathBookLaunch     2514815   
                      #Masters2016                2514815   
                      #NationalSiblingsDay        2514815   
                      #PacBradley                 2514815   
                      #ResistCapitalism           2514815   
           2016-04-11 #BBMAs                      2514815   
                      #Hearties                   2514815   
                      #mondaymotivation           2514815   
                      Spurs                       2514815   
                      Warriors                    2514815   
                      Will Smith                  2514815   
           2016-04-12 #HalaMadrid                 2514815   
                      #LHHATL                     2514815   
                      #NationalPetDay             2514815   
                      #SultanTeaser               2514815   
           2016-04-13 #CaptainAmericaPremiere     2514815   
                      #HalaMadrid                 2514815   
                      #MambaDay                   2514815   
                      Barca                       2514815   
                      Doctor Strange              2514815   
                      Ronaldo                     2514815   
                      Sony                        2514815   
           2016-04-14 #Empire                     2514815   
                      #ISharedTheVamps            2514815   
                      Barcelona                   2514815   
                      Kobe                        2514815   
                      Rams                        2514815   
           2016-04-15 #6YearsSinceNiallsAudition  2514815   
                      #BuyHairOniTunes            2514815   
                      #DemDebate                  2514815   
                      #UCLDraw                    2514815   
                      Kobe                        2514815   
                      Kumamoto                    2514815   
                      Liverpool                   2514815   
                      Prince                      2514815   
           2016-04-16 #BernieAtTheVatican         2514815   
                      #CONGRATULATIONSHARRY       2514815   
                      #Coachella                  2514815   
                      #MUFC                       2514815   
                      Coachella                   2514815   
                      Getafe                      2514815   
                      Japan                       2514815   
                      Kobe                        2514815   
                      Pope                        2514815   
                      Prince                      2514815   
                      getafe                      2514815   
           2016-04-17 #Ecuador                    2514815   
                      Ecuador                     2514815   

                                                                                          searchUrl  \
placeName  trendDate  trendName                                                                       
Washington 2016-04-09 Bruce Springsteen           http://twitter.com/search?q=%22Bruce+Springste...   
                      Trevor Story                   http://twitter.com/search?q=%22Trevor+Story%22   
           2016-04-10 #EverydayKathBookLaunch     http://twitter.com/search?q=%23EverydayKathBoo...   
                      #Masters2016                       http://twitter.com/search?q=%23Masters2016   
                      #NationalSiblingsDay        http://twitter.com/search?q=%23NationalSibling...   
                      #PacBradley                         http://twitter.com/search?q=%23PacBradley   
                      #ResistCapitalism             h

In [83]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

print_full(trends_agg_322.index.get_level_values("placeName").value_counts())

#trends_agg_322["trendMapped"].value_counts()

New York            274
Albuquerque         273
United States       272
Fresno              272
Long Beach          271
Tucson              270
Philadelphia        267
San Jose            267
Mesa                266
Colorado Springs    265
Los Angeles         264
Washington          256
Virginia Beach      255
San Francisco       251
Chicago             242
Boston              242
Atlanta             238
Dallas-Ft. Worth    229
Houston             217
Miami               216
Denver              214
Cleveland           210
Seattle             205
Tampa               203
Phoenix             197
Minneapolis         195
San Diego           195
Detroit             190
Las Vegas           189
St. Louis           189
New Haven           189
Charlotte           188
Indianapolis        183
Salt Lake City      182
Austin              182
Raleigh             182
Orlando             177
Louisville          177
Portland            175
Baton Rouge         174
Columbus            174
Pittsburgh      

Here we get the mapped names for giphy retrieval

In [123]:
mike = [ dude.replace(" ", "+") for dude in list(set(trends_agg_322["trendMapped"]))]
responses = []

In [124]:
for trend in mike:
    resp = requests.get("http://api.giphy.com/v1/gifs/search?q=" + trend + "&api_key=dc6zaTOxFJmzC&limit=1")
    data = resp.json()
    data["trend"] = trend
    responses.append(data)
    

In [138]:
for resp in responses:
    if resp["data"]:
        url = resp["data"][0]["images"]["downsized_large"]["url"]
        res = requests.get(url, stream=True)
        with open("gifs_full/"+resp["trend"]+'.gif', 'wb') as out_file:
            shutil.copyfileobj(res.raw, out_file)
        del res

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [88]:
dude = pd.Series(["NY","NM","XX","CA","CA","AZ","PA","CA","AZ","CO",
                  "CA","DC","VA","CA","IL","MA","GA","TX","TX","FL",
                  "CO","OH","WA","FL","AZ","MN","CA","MI","NV","MO",
                  "CT","NC","IN","UT","TX","NC","FL","KY","OR","LA",
                  "OH","PA","MD","TX","NE","OK","MS","TX","WI","TN",
                  "OH","RI","FL","VA","MO","VA","PA","CA","AL","TN",
                  "HI","NC","FL","LA"])

In [133]:
responses[0].keys()

['trend', u'pagination', u'meta', u'data']

u'taksim'